# DATA CLEANING NOTEBOOK
Felix A. Westphal
DLMDWME01

### Import

In [30]:
import pandas as pd
from datetime import datetime

### Parameter

In [31]:
FILE_PATH_RAW_DATA = r"../data/raw/PSP_Jan_Feb_2019.xlsx"
FILE_PATH_CLEANED_DATA = r"../data/processed/Cleaned_Input_Data.csv"
TIMESTAMP_FORMAT = "%Y-%m-%d %H:%M:%S"

### Value Types and Ranges

In [32]:
COLUMN_HEADERS = ["tmsp", "country", "amount", "success", "PSP", "3D_secured", "card"]
VALUE_RANGE_TMSP = [datetime.strptime("2010-01-01 00:00:00", TIMESTAMP_FORMAT), datetime.now()]
VALUE_RANGE_AMOUNT = [0.0, None]
VALUE_RANGE_PSP = ["Moneycard", "Goldcard", "UK_Card", "Simplecard"]
VALUE_RANGE_CARD = ["Master", "Visa", "Diners"]
DEFINED_TYPES = ["datetime64[ns]", "string", "float32", bool, "string", bool, "string"]
VALUE_RANGES = [VALUE_RANGE_TMSP, None, VALUE_RANGE_AMOUNT, None, VALUE_RANGE_PSP, None, VALUE_RANGE_CARD]
value_range_dict = {k : {"type" : v1, "range" : v2} for k, v1, v2 in zip(COLUMN_HEADERS, DEFINED_TYPES, VALUE_RANGES)}

### Load Raw Data and remove NAN values

In [33]:
raw_data = pd.read_excel(FILE_PATH_RAW_DATA, index_col=0)       # Load raw data file
print(f"Raw Data loaded from Excel: \n{raw_data}")
sum_nan_column = raw_data.isna().sum()                          # Check for nan values
print(f"Number of NAN values per column: \n{sum_nan_column}")
total_sum_nan = sum_nan_column.sum()
print(f"Total number of NAN values: {total_sum_nan}")
if total_sum_nan > 0:
    filtered_df = raw_data.dropna()
    print(f"Filtered Data Frame: \n{filtered_df}")
else:
    filtered_df = raw_data

Raw Data loaded from Excel: 
                     tmsp      country  amount  success         PSP  \
0     2019-01-01 00:01:11      Germany      89        0     UK_Card   
1     2019-01-01 00:01:17      Germany      89        1     UK_Card   
2     2019-01-01 00:02:49      Germany     238        0     UK_Card   
3     2019-01-01 00:03:13      Germany     238        1     UK_Card   
4     2019-01-01 00:04:33      Austria     124        0  Simplecard   
...                   ...          ...     ...      ...         ...   
50405 2019-02-28 23:45:39  Switzerland     415        0     UK_Card   
50406 2019-02-28 23:46:48      Austria      91        0     UK_Card   
50407 2019-02-28 23:47:04      Austria      91        0     UK_Card   
50408 2019-02-28 23:47:36      Austria      91        0     UK_Card   
50409 2019-02-28 23:48:19      Austria      91        1   Moneycard   

       3D_secured    card  
0               0    Visa  
1               0    Visa  
2               1  Diners  
3     

### Check valid Value Range

In [34]:
print(f"\n========== CHECKING VALUE RANGES AND TYPES ==========")
# --- Loop through coloumns and check whether datatype is as defined, change if not.
num_rows_filtered = len(filtered_df.index)
for column_header in COLUMN_HEADERS:
    print(f"--- Current Column: {column_header} ---")
    column = filtered_df[column_header]
    defined_type = value_range_dict[column_header]["type"]
    current_type = column.dtype
    print(f"Current Type / Defined Type: {current_type} / {defined_type}")
    if current_type != defined_type:
        if defined_type == datetime:
            column = pd.to_datetime(column, errors="coerce", format=TIMESTAMP_FORMAT)
        elif defined_type == float:
            column = pd.to_numeric(column, errors="coerce", downcast=defined_type)
        else:
            column = column.astype(defined_type)
        current_type = column.dtype
        print(f"Changed type to {current_type}")
    filtered_df[column_header] = column

    # --- Check whether values are within defined value range.
    defined_value_range = value_range_dict[column_header]["range"]
    if defined_value_range:
        if len(defined_value_range) == 2:
            min_value_defined = defined_value_range[0]
            max_value_defined = defined_value_range[1]
            if min_value_defined:
                min_value = min(column)
                print(f"Min Value / Defined min Value: {min_value} / {min_value_defined}")
                if min_value < min_value_defined:
                    print(f"Found entry with smaller value than defined. Removing entry!")
                    filtered_df = filtered_df[filtered_df[column_header] >= min_value_defined]
            if max_value_defined:
                max_value = max(column)
                print(f"Max Value / Defined max Value: {max_value} / {max_value_defined}")
                if max_value > max_value_defined:
                    print(f"Found entry with larger value than defined. Removing entry!")
                    filtered_df = filtered_df[filtered_df[column_header] <= max_value_defined]
        else:
            unique_values = list(pd.unique(filtered_df[column_header]))
            not_defined_values = [value for value in unique_values if value not in defined_value_range]
            if len(not_defined_values) > 0:
                filtered_df = filtered_df[~filtered_df[column_header].isin(not_defined_values)]             # Remove all columns which are outside the defined range

cleaned_df = filtered_df.dropna()
num_rows_cleaned = len(cleaned_df.index)
num_rows_deleted = num_rows_filtered - num_rows_cleaned
print(f"\n--- Deleted {num_rows_deleted} rows from Filtered Data Frame. ---")
print(f"\n========== CLEANED DATA FRAME ========== \n{filtered_df}")
cleaned_df.to_csv(FILE_PATH_CLEANED_DATA, index=False)
print(f"Saved Cleaned Data Frame as .csv to: {FILE_PATH_CLEANED_DATA}")


========== CHECKING VALUE RANGES AND TYPES ==========
--- Current Column: tmsp ---
Current Type / Defined Type: datetime64[ns] / datetime64[ns]
Min Value / Defined min Value: 2019-01-01 00:01:11 / 2010-01-01 00:00:00
Max Value / Defined max Value: 2019-02-28 23:48:19 / 2023-04-29 10:36:32.139800
--- Current Column: country ---
Current Type / Defined Type: object / string
Changed type to string
--- Current Column: amount ---
Current Type / Defined Type: int64 / float32
Changed type to float32
--- Current Column: success ---
Current Type / Defined Type: int64 / <class 'bool'>
Changed type to bool
--- Current Column: PSP ---
Current Type / Defined Type: object / string
Changed type to string
--- Current Column: 3D_secured ---
Current Type / Defined Type: int64 / <class 'bool'>
Changed type to bool
--- Current Column: card ---
Current Type / Defined Type: object / string
Changed type to string

--- Deleted 0 rows from Filtered Data Frame. ---

========== CLEANED DATA FRAME ========== 
   